In [87]:
import fastf1
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
from math import exp
import os
from tqdm import tqdm

#### Weather & Performance Correlation
Quantify how temperature, humidity & wind direction affect car performance
Build a sensitivity surface for each constructor with visuals

In [88]:
import logging
logging.getLogger("fastf1").setLevel(logging.CRITICAL)

In [89]:
os.makedirs('cache', exist_ok=True)
fastf1.Cache.enable_cache('cache')

In [90]:
# Fetch session
session = fastf1.get_session(2024, 'Silverstone', 'R')
session.load(weather=True, telemetry=False)

laps = session.laps
weather = session.weather_data  # time-indexed weather info
# Convert weather time (timedelta since session start) to absolute timestamp
session_start = session.session_start_time
weather['AbsTime'] = session_start + weather['Time']

In [91]:
# Extract & merge data
df = pd.merge_asof(
    laps.sort_values('LapStartTime'),
    weather.assign(AbsTime=session_start + weather['Time']).sort_values('AbsTime'),
    left_on='LapStartTime', right_on='AbsTime',
    direction='nearest', tolerance=pd.Timedelta('1min')
)[['Driver', 'Team', 'LapTime', 'AirTemp', 'Humidity', 'TrackTemp']]

df['LapTime_s'] = df['LapTime'].dt.total_seconds()
df.head()

,Driver,Team,LapTime,AirTemp,Humidity,TrackTemp,LapTime_s
0,HAM,Mercedes,0 days 00:01:36.034000,15.9,67.0,28.1,96.034
1,PIA,McLaren,0 days 00:01:38.008000,15.9,67.0,28.1,98.008
2,SAI,Ferrari,0 days 00:01:38.699000,15.9,67.0,28.1,98.699
3,HUL,Haas F1 Team,0 days 00:01:40.166000,15.9,67.0,28.1,100.166
4,STR,Aston Martin,0 days 00:01:39.108000,15.9,67.0,28.1,99.108


In [92]:
# Muliple races
races = [
    ('Bahrain', 'R'),
    ('Jeddah', 'R'),
    ('Melbourne', 'R'),
    ('Miami', 'R'),
    ('Barcelona', 'R'),
    ('Silverstone', 'R'),
    ('Hungaroring', 'R'),
    ('Monza', 'R'),
    ('Singapore', 'R'),
    ('Austin', 'R')
]

all_laps = []

for circuit, session_type in tqdm(races):
    try:
        s = fastf1.get_session(2024, circuit, session_type)
        s.load(weather=True, telemetry=False)

        laps = s.laps[['Driver', 'Team', 'LapTime', 'LapStartTime']].copy()
        weather = s.weather_data.copy()
        weather['AbsTime'] = s.session_start_time + weather['Time']

        merged = pd.merge_asof(
            laps.sort_values('LapStartTime'),
            weather.sort_values('AbsTime'),
            left_on='LapStartTime', right_on='AbsTime',
            direction='nearest', tolerance=pd.Timedelta('1min')
        )

        merged['LapTime_s'] = merged['LapTime'].dt.total_seconds()
        merged['Circuit'] = circuit
        all_laps.append(
            merged[['Circuit', 'Team', 'Driver', 'LapTime_s', 'AirTemp', 'Humidity', 'TrackTemp']]
        )
    except Exception as e:
        print(f"Exception: {circuit}: {e}")

df_all = pd.concat(all_laps, ignore_index=True).dropna(subset=['AirTemp'])
df_all.head()

100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


,Circuit,Team,Driver,LapTime_s,AirTemp,Humidity,TrackTemp
0,Bahrain,Red Bull Racing,VER,97.284,18.9,46.0,26.5
1,Bahrain,Ferrari,LEC,98.271,18.9,46.0,26.5
2,Bahrain,Mercedes,RUS,99.228,18.9,46.0,26.5
3,Bahrain,McLaren,NOR,102.168,18.9,46.0,26.5
4,Bahrain,Mercedes,HAM,103.122,18.9,46.0,26.5
